# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv('../output_data/CityWeather.csv')

In [3]:
df = df.rename(columns = {'Unnamed: 0' : 'City ID' })
df.head()
df['Lat'].count()

554

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key = g_key)

In [5]:
fig = gmaps.figure(center = (45.0, -2.0), zoom_level = 1.8)
locations = df[["Lat", "Lng"]]
humidity = df["Humidity"]



heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating = False, max_intensity = 100,
                                 point_radius = 4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Nice weather at max temperature lower than 80 degrees but higher than 70.
new_df = df.loc[(df['Max. Temp.'] < 80) & (df['Max. Temp.'] > 70 )]
# Not too windy cities
new_df = new_df.loc[(new_df['Wind Speed'] < 10)]
# Ideally humidity 
new_df = new_df.loc[(new_df['Humidity'] < 60)]
# Nice clear sky cities
new_df = new_df.loc[(new_df['Cloudiness'] == 0)]
# Drop any rows will null values.
new_df = new_df.dropna().reset_index()
print('Best cities for vacation are:')
print('*******************')
print(f'{new_df.City}')
print('*******************')

Best cities for vacation are:
*******************
0         Hobart
1      Carnarvon
2    Meadow Lake
3          Ubari
4     Abu Samrah
5        Corumbá
6         Kirkuk
Name: City, dtype: object
*******************


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Store into variable named hotel_df
hotel_df = new_df.reset_index().drop( columns = ['level_0', 'index'])
# Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ''
hotel_df

,City ID,City,Country,Lat,Lng,Date,Cloudiness,Max. Temp.,Humidity,Wind Speed,Hotel Name
0,49,Hobart,AU,-42.8794,147.3294,1632273687,0,70.11,50,5.99,
1,68,Carnarvon,AU,-24.8667,113.6333,1632273951,0,78.87,19,5.75,
2,88,Meadow Lake,US,34.8014,-106.5436,1632273969,0,74.30,32,4.07,
3,349,Ubari,LY,26.5921,12.7805,1632274141,0,79.59,18,9.60,
4,443,Abu Samrah,SY,35.3029,37.1841,1632274210,0,71.58,46,6.69,
5,491,Corumbá,BR,-19.0092,-57.6533,1632274243,0,76.89,59,7.34,
6,509,Kirkuk,IQ,35.4681,44.3922,1632274253,0,76.71,23,0.85,


In [8]:
# define url for API calls
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
params = {"types" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

# Hit the Google Places API for each city's coordinates
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # make request and print url
    hotel_name = requests.get(base_url, params = params)
    
    # convert to json format
    hotel_name = hotel_name.json()

    # Store the first Hotel result into the DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")    
        
# Plot markers on top of the heatmap

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))